In [0]:
%reload_ext autotime
FORECAST_TERM_CODE = 202508
LAG = 1
exec(open('./amp.py').read())

aggregates = [
    'styp_desc',
    'camp_desc',
    'coll_desc',
    'dept_desc',
    'majr_desc',
    'hs_qrtl',
    'tsi_math',
    'tsi_reading',
    'tsi_writing',
    # 'gender',
    # *races,
    # 'international',
    'resd_desc',
    'oriented',
    'waiver',
    'lgcy',
]

features = {
    'act_equiv':pd.NA,
    'age':pd.NA,
    'camp_desc':'stephenville',
    'drivetime':pd.NA,
    'eager':pd.NA,
    'fafsa': False,
    'finaid': False,
    'gap_score':0,
    'gender':pd.NA,
    'hs_qrtl':pd.NA,
    'international':False,
    'lgcy':False,
    'oriented':False,
    **{r: False for r in races},
    'schlship':False,
    'ssb':False,
    'tsi_math':False,
    'tsi_reading':False,
    'tsi_writing':False,
    'verified':False,
    'waiver':False,
}

crse_codes = [
    '_headcnt',
    '_proba',
    'agec2317','ansc1119','ansc1319','anth2302','anth2351','arts1301','arts1303','arts1304','arts3331',
    'biol1305','biol1406','biol1407','biol2401','biol2402','busi1301','busi1307','chem1111','chem1112',
    'chem1302','chem1311','chem1312','chem1407','chem1409','cnst1301','comm1311','comm1315','comm2302',
    'crij1301','dram1310','dram2361','easc2310','econ1301','econ2301','educ1301','engl1301','engl1302',
    'engl2307','engl2320','engl2321','engl2326','engl2340','engl2350','engl2360','engl2362','engl2364',
    'engl2366','engl2368','engr1211','engr2303','envs1302','fina1360','geog1303','geog1320','geog1451',
    'geog2301','geol1403','geol1404','geol1407','geol1408','govt2305','govt2306','hist1301','hist1302',
    'hist2321','hist2322','huma1315','kine1301','kine1338','kine2315','math1314','math1316','math1324',
    'math1325','math1332','math1342','math1352','math2412','math2413','musi1303','musi1310','musi1311',
    'musi2350','musi3325','phil1301','phil1304','phil2303','phil3301','phys1302','phys1401','phys1402',
    'phys1403','phys1411','phys2425','phys2426','psyc2301','psyc3303','psyc3307','soci1301','soci1306',
    'soci2303','univ0010','univ0200','univ0204','univ0301','univ0314','univ0324','univ0332','univ0342',
]

REFRESHED = []
REFRESH = [
    # 'flags',
    # 'admissions',
    # 'registrations',
    # 'students',
    # 'imputed',
    # 'learners',
    # 'predictions',
    # 'enrollments',
    # 'enrollmentsall',
    # 'studentsall',
    # 'predictionsall',
    # 'proba',
    # 'forecasts',
]

kwargs = {'features': features, 'aggregates':aggregates, 'crse_codes':crse_codes, 'start':2022,
    # 'date': '2025-08-13',
    'flaml':{
        'time_budget': 60,
    }}
# kwargs |= {'crse_codes':['_headcnt','_probb'], 'aggregates':['camp_desc']}  #quick for testing
if LAG == 1:
    kwargs |= {'crse_codes':'_proba', 'aggregates':[]}  #retention
self = AMP(**kwargs)
# self.setup()
# self.get_flagsday() # GA's should not have permissions to run this because it sees pii
self.get_proba()
# self.get_forecasts()
# self.get_reports() # may hit memory limits running get_flagsyear - just restart kernel and try again

In [0]:
A['prediction_term'].unique()

In [0]:
from sklearn.metrics import log_loss
from sklearn.metrics import RocCurveDisplay
# def log_loss(y_true, y_pred):
#     return -1 * (y_true * np.log(1-y_pred) + (1-y_true) * np.log(1-y_pred))
A = self.get_proba()#.query()
# t = P['stable']
# p = P['prediction']
# P['log_loss'] = -1 * (t*np.log(p) + (1-t)*np.log(1-p))
# P['log_loss'] = log_loss(P['stable'], P['prediction'])
# P.groupmy(union(self.subpops))['log_loss'].mean()

import matplotlib.pyplot as plt


for pred_term, B in A.groupmy('prediction_term', sort=True):
    fig, ax = plt.subplots()
    for modl_term, C in B.groupmy('model_term', sort=True):
        RocCurveDisplay.from_predictions(C['stable'], C['prediction'], ax=ax, name=modl_term)
    ax.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
    ax.set_title(pred_term)
    plt.show()

# for k, v in P.groupmy(['prediction_term', 'model_term']):
#     print(log_loss(P['stable'], P['prediction']))
#     RocCurveDisplay.from_predictions(v['stable'], v['prediction'], ax=ax, name=k)#.plot(ax=ax, label=k)
# ax.plot([0, 1], [0, 1], 'k--', label='Random Classifier')
# plt.show()

# log_loss(P['stable'], P['prediction'])


In [0]:
dct = {join(s): df for s, df in self.get_spriden().merge(self.get_proba().query(f"prediction_term==@self.term_desc & model_term==model_term.max()"), how='right').groupmy(self.subpops)}

In [0]:
dct['new first time']

In [0]:
A = self.get_spriden().merge(self.get_proba(), how='right')
B = A.query(f"prediction_term==@self.term_desc & model_term==model_term.max()")#.loc[:,:'prediction'].query("pidm==1000)

In [0]:
A = run_qry(f"select * except (crse_code, cv_score, mlt) {self.subqry} where crse_code='_proba' limit 10")
# move(A, 'model_term', 'prediction_term')
A.move('model_term', 'prediction_term').move('stable', 'pidm')
A

In [0]:
dct = {join(s): df for s, df in self.get_spriden().merge(self.get_proba(), how='right').groupmy(self.subpops)}

In [0]:
self.setup()

In [0]:
self.subqry

In [0]:
union('pidm', self.subpops, 'prediction_term', 'model_term',' prediction', 'stable')

In [0]:
def move(self, src, dst, offset=1):
    self.insert(self.columns.get_loc(dst) + offset, src, self.pop(src))
    return self
A = run_qry(f"select * except (crse_code, cv_score, mlt) {self.subqry} where crse_code='_proba' limit 10")
move(A, 'model_term', 'prediction_term')

In [0]:
A = run_qry(f"select * except (crse_code, cv_score, mlt) {self.subqry} where crse_code='_proba' limit 10")
A.insert(A.columns.get_loc('prediction_term') + 1, 'model_term', A.pop('model_term'))
A

In [0]:
A

In [0]:
# run_qry(f"select * except (crse_code, cv_score, mlt) {self.subqry} where crse_code='_proba' limit 10")
run_qry(f"select pidm, * except (crse_code, cv_score, mlt) {self.subqry} where crse_code='_proba' limit 10")

In [0]:
self.get_proba()

In [0]:
dct['new first time'].
C = self.run_predictions('_proba').loc[('_proba',self.term_code,self.term_code-100),'prediction']

In [0]:
rpts = {
    # 'summary':lambda df: df.query(f"prediction_term==@self.term_desc").loc[:,:'prediction'],
    # 'details':lambda df: df,
}
rpts = {rpt: {agg: fcn(df).round().prep() for agg, df in self.get_forecasts().items()} for rpt, fcn in rpts.items()}
rpts['em'] = {s: df for s, df in self.get_spriden().merge(self.get_proba(), how='right').groupmy(self.subpops)}

def format_xlsx(sheet):
    from openpyxl.styles import Alignment
    sheet.auto_filter.ref = sheet.dimensions
    for cell in sheet[1]:
        cell.alignment = Alignment(horizontal="left")
    for column in sheet.columns:
        width = 1+max(len(str(cell.value))+3*(i==0) for i, cell in enumerate(column))
        sheet.column_dimensions[column[0].column_letter].width = width
    sheet.freeze_panes = "A2"

def write_xlsx(rpt, sheets):
    nm, dst = self.get_dst(f'reports/{self.date}/{rpt}', suffix='.xlsx')
    reset(dst)
    print(f'creating {dst}', end=': ')
    with codetiming.Timer():
        src = "./report.xlsx"
        # sheets = {'instructions':instructions} | sheets
        with pd.ExcelWriter(src, mode="w", engine="openpyxl") as writer:
            for sheet_name, df in sheets.items():
                df.to_excel(writer, sheet_name=join(sheet_name), index=False)
                format_xlsx(writer.sheets[sheet_name])
        shutil.copy(src, dst)
        rm(src)
    print(divider)

for rpt, sheets in rpts.items():
    write_xlsx(rpt, sheets)

In [0]:
[join(k, ', ') for k in sheets.keys()]

In [0]:
rpts['em'][('new first time',)]